# Extracting Building Values from an API

In [1]:
import pandas as pd
import json
import requests 
import zipcodes
import http.client 
import matplotlib.pyplot as plt
from collections import defaultdict 

## [ATTOM API Docs](https://api.developer.attomdata.com/docs)

In [2]:
# Input the desired zipcode
while True: 
    zipinput = input("Enter a zip code value ")
    
    if zipcodes.is_real(zipinput):
        
        break
        
    else:
        print("Invalid zip code...re-enter zip code")


Enter a zip code value 78701


In [3]:
# Input the date of the desired valuations
while True:
    year = input("Enter the current year in 4 digits: ")
    if len(year)==4:
        year=int(year)
        break
    else:
        print("Invalid Year. Please Enter the Year as 4-digits.")
        
while True:
    month = input("Enter the current month in 2 digits: ")
    if len(month)==2:
        month=int(month)
        break
    else:
        print("Invalid Year. Please Enter a Zero before a 1-digit month.")
        
while True:
    day = input("Enter the current day in 2 digits: ")
    if len(day)==2:
        day=int(day)
        break
    else:
        print("Invalid Date. Please Enter a Zero before a 1-digit day.")


Enter the current year in 4 digits: 2020
Enter the current month in 2 digits: 02
Enter the current day in 2 digits: 17


In [4]:
# Pull data on property values from the previous year from the given date, from the ATTOM Data API
with open('../../misc/api_keys/attom_a.txt') as f:
    key = f.read().strip()

conn = http.client.HTTPSConnection("api.gateway.attomdata.com") 
headers = { 
    'accept': "application/json", 
    'apikey': key
    } 
conn.request("GET",f"/propertyapi/v1.0.0/assessment/detail?geoid=ZI{zipinput}&startcalendardate={year-1}-{month}-{day}&endcalendardate={year}-{month}-{day}&page=1&pagesize=10000", headers=headers)

res = conn.getresponse() 
data = res.read() 

dict1 = json.loads(data.decode("utf-8"))

FileNotFoundError: [Errno 2] No such file or directory: '../misc/api_keys/attom_a.txt'

In [ ]:
dict1['property'][0].keys()  #all the keys for the first property in the list

In [ ]:
pd.DataFrame(dict1['property'][0]) # the values for the first property in the list

In [ ]:
dict1['property'][0]['assessment']['assessed']

In [ ]:
dict1['property'][0]['assessment']['appraised']

In [ ]:
dict1['property'][0]['assessment']['calculations']

In [ ]:
dict1['property'][0]['assessment']['market']

In [ ]:
df_property = pd.DataFrame({
    'zip_input': [zipinput for num in range((len(dict1['property'])))],
    'zipcode': [dict1['property'][num]['address']['postal1'] for num in range((len(dict1['property'])))],
    'state': [dict1['property'][num]['address']['countrySubd'] for num in range((len(dict1['property'])))],
    'address': [dict1['property'][num]['address']['line1'] for num in range((len(dict1['property'])))],
    'property_type': [dict1['property'][num]['summary']['proptype'] for num in range((len(dict1['property'])))],
    'property_code': [dict1['property'][num]['summary']['propIndicator'] for num in range((len(dict1['property'])))],
    'city': [dict1['property'][num]['address']['locality'] for num in range((len(dict1['property'])))],
    'latitude': [dict1['property'][num]['location']['latitude'] for num in range((len(dict1['property'])))],
    'longitude': [dict1['property'][num]['location']['longitude'] for num in range((len(dict1['property'])))],
    'sqft': [dict1['property'][num]['lot']['lotsize1'] for num in range((len(dict1['property'])))],
    'appraised_improved_value': [dict1['property'][num]['assessment']['appraised']['apprimprvalue'] for num in range((len(dict1['property'])))],
    'appraised_land_value': [dict1['property'][num]['assessment']['appraised']['apprlandvalue'] for num in range((len(dict1['property'])))],
    'appraised_total_value': [dict1['property'][num]['assessment']['appraised']['apprttlvalue'] for num in range((len(dict1['property'])))],
    'assessed_improved_value': [dict1['property'][num]['assessment']['assessed']['assdimprvalue'] for num in range((len(dict1['property'])))],
    'assessed_land_value': [dict1['property'][num]['assessment']['assessed']['assdlandvalue'] for num in range((len(dict1['property'])))],
    'assessed_total_value': [dict1['property'][num]['assessment']['assessed']['assdttlvalue'] for num in range((len(dict1['property'])))],
    'calculated_improved_value': [dict1['property'][num]['assessment']['calculations']['calcimprvalue'] for num in range((len(dict1['property'])))],
    'calculated_land_value': [dict1['property'][num]['assessment']['calculations']['calclandvalue'] for num in range((len(dict1['property'])))],
    'calculated_total_value': [dict1['property'][num]['assessment']['calculations']['calcttlvalue'] for num in range((len(dict1['property'])))],
    'market_value': [dict1['property'][num]['assessment']['market']['mktttlvalue'] for num in range((len(dict1['property'])))],
})

print(df_property.shape)
print(f'There are {df_property.shape[0]} properties found in the {zipinput} zipcode for the year preceding {month}/{day}/{year}.')
df_property.head()

In [ ]:
# Data Dictionary for property types
# """Property Type - Codes
# 10 Single Family Residence / Townhouse/
# 11 Condominium (residential)/
# 20 Commercial
# 21 Duplex, Triplex, Quadplex/
# 22 Apartment/
# 23 Hotel, Motel/
# 24 Commercial (condominium)
# 25 Retail/
# 27 Office Building/
# 28 Warehouse/
# 29 Financial Institution/
# 30 Hospital (medical complex, clinic)
# 50 Industrial/
# 51 Industrial Light/
# 52 Industrial Heavy/
# 54 Utilities
# 70 Agricultural/
# 80 Vacant/
# 90 Exempt"""


In [ ]:
value_cols = list(df_property.columns[df_property.columns.str.contains('value')])
value_cols

In [ ]:
final = {}
final['zipcode'] = 78701
final['count_property'] = df_property['address'].count()

for col in value_cols:
    final[f'sum_{col}'] = 0
    for value in df_property[col]:
#         print(value)
        final[f'sum_{col}'] += value
    final[f'avg_{col}'] = round(final[f'sum_{col}']/len(df_property[col]),2)
pd.DataFrame([final])

In [ ]:
def get_custom_describe(value_col, df_group=df_property.groupby(['property_type'])):
    return pd.DataFrame({
                        'count': df_group[value_col].count(),
                        'mean': df_group[value_col].mean().map('${:,.2f}'.format),
                        'std': df_group[value_col].std().map('${:,.2f}'.format),
                        'median_val': df_group[value_col].median().map('${:,.2f}'.format),
                        'min_val': df_group[value_col].min().map('${:,.2f}'.format),
#                         '25_percentile': df_group[value_col].quantile(.25).map('${:,.2f}'.format),
#                         '75_percentile': df_group[value_col].quantile(.75).map('${:,.2f}'.format),
                        'max_val': df_group[value_col].max().map('${:,.2f}'.format),
                        'max_total': df_group[value_col].sum().map('${:,.2f}'.format)

    })

In [ ]:
df = get_custom_describe(df_group=df_property, value_col=['calculated_land_value', 'calculated_improved_value', 
                                                      'calculated_total_value'])
df = df.T.copy()
df.drop(['count', 'std'], inplace=True)
df.rename(columns={
    'calculated_land_value': 'Land Value', 
    'calculated_improved_value': 'Improved Value On the Land', 
    'calculated_total_value': 'Total Property Value'
}, inplace=True)
df.rename({
    'mean': 'Mean Property Value',
    'min_val': 'Minimum Property Value',
    'median_val': 'Median Property Value',
    'max_val': 'Maximum Property Value',
    'max_total': 'TOTAL PROPERTIES VALUE'  
}, inplace=True)
df

In [ ]:
get_custom_describe(value_col='calculated_improved_value')

In [ ]:
get_custom_describe(value_col='calculated_land_value')

In [ ]:
get_custom_describe(value_col='calculated_total_value') # total value includes value of the land 

In [ ]:
for row in df_property.T:
    if df_property['calculated_total_value'][row] != df_property['market_value'][row]:
        print(row)
#     else:
#         print('It is the same value.')

In [ ]:
ax = df_property.groupby(['property_type'])['calculated_land_value', 
                                            'calculated_improved_value'].sum().plot.barh(figsize=(10,10), 
                                                                                         stacked=True)
ax.set_ylabel('Property Type', fontsize=15)
ax.set_title('Total Land and Combined Property Values by Property Type', fontsize=15)
ax.legend(('Total Land Value', 'Total Improved Value'))
;
plt.savefig(f'./images/stacked_property_val_chart_{zipinput}_{year}_{month}_{day}.png')

In [ ]:
# Pull Sales time series data from the ATTOM API
conn2 = http.client.HTTPSConnection("api.gateway.attomdata.com") 
conn2.request("GET",f"/propertyapi/v1.0.0/salestrend/snapshot?geoid=ZI{zipinput}&interval=yearly&startyear=1979&endyear={year}", headers=headers)
res2 = conn2.getresponse() 
data2 = res2.read() 

dict2 = json.loads(data2.decode("utf-8"))

In [ ]:
price_series = defaultdict(list)          #creates dictionary that's empty made of lists


#Build list of properties & key data
for price in dict2['salestrends']:
    price_series['location'].append(price['location']['geoID'])
    price_series['year'].append(price['daterange']['start'])
    price_series['SF_count'].append(price['SalesTrend']['homesalecount'])
    price_series['AvgSalePrice'].append(price['SalesTrend']['avgsaleprice'])
    price_series['MedianSalePrice'].append(price['SalesTrend']['medsaleprice'])
df = pd.DataFrame(price_series)

In [ ]:

plt.figure(figsize=(10,8))
plt.title('Sales Trends for Single Family Homes in '+zipinput, fontsize=20)

for y_col in ['AvgSalePrice', 'MedianSalePrice', 'SF_count']:  
    plt.plot(df['year'], df[y_col])
    
plt.legend(labels=('Average Price', 'Median Price', 'Sales Count'), fontsize=20);
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig(f'./images/sales_trends_graph_{zipinput}_{year}_{month}_{day}.png')
plt.show();
